In [2]:
# %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import butter, filtfilt, freqz
from scipy.signal import argrelextrema

In [10]:
pos = np.loadtxt('../data/Position_data/PosFiles/Mouse12-120807.txt')
# pos = np.loadtxt('../data/Position_data/PosFiles/Mouse12-120810.txt')
# pos = np.loadtxt('../data/Position_data/PosFiles/Mouse17-130202.txt')

In [11]:
xx_blue = pos[:,0]
yy_blue = pos[:,1]

wind_b, wind_e = 5698.9,7778.8
fr = 39.06
wind_point_b,wind_point_e = int(wind_b*fr), int(wind_e*fr)

In [12]:
traj_b = xx_blue + 1j * yy_blue
traj_b[xx_blue==-1] = np.nan

traj_b = traj_b[wind_point_b:wind_point_e]

accel = np.abs(traj_b[:-1:1] - traj_b[1::1] )
accel = accel[1:]

plt.plot( accel )
plt.show()

In [13]:
def savitzky_golay(y, window_size, order, deriv=0):
    try:
        window_size = np.abs(np.int(window_size))
        order = np.abs(np.int(order))
    except ValueError, msg:
        raise ValueError("window_size and order have to be of type int")
    if window_size % 2 != 1 or window_size < 1:
        raise TypeError("window_size size must be a positive odd number")
    if window_size < order + 2:
        raise TypeError("window_size is too small for the polynomials order")
    order_range = range(order+1)
    half_window = (window_size -1) // 2
    # precompute coefficients
    b = np.mat([[k**i for i in order_range] for k in range(-half_window, half_window+1)])
    m = np.linalg.pinv(b).A[deriv]
    # pad the signal at the extremes with
    # values taken from the signal itself
    firstvals = y[0] - np.abs( y[1:half_window+1][::-1] - y[0] )
    lastvals = y[-1] + np.abs(y[-half_window-1:-1][::-1] - y[-1])
    y = np.concatenate((firstvals, y, lastvals))
    return np.convolve( m, y, mode='valid')

In [14]:
smooth_accel = savitzky_golay( accel , 51, 2) # window size 51, polynomial order 3

plt.plot( accel  )
plt.plot( smooth_accel, 'r' , linewidth=3  )
plt.show()

In [15]:
local_max = argrelextrema(smooth_accel, np.greater)[0] # for local maxima
local_min = argrelextrema(smooth_accel, np.less)[0] # for local minima

prox_indx = np.where( local_min[1:]-local_min[:-1] > 10 )[0]
prox_indx_valid = local_min[prox_indx]

local_range = 20
lmin_indx = np.where( smooth_accel[prox_indx_valid+local_range][:-local_range] - smooth_accel[prox_indx_valid][:-local_range]  > 0.1   )[0]
lmin_indx_valid = prox_indx_valid[lmin_indx]

plt.plot( smooth_accel, 'r' , linewidth=3  )
plt.vlines(  lmin_indx_valid , np.zeros(len(lmin_indx_valid)) , np.ones(len(lmin_indx_valid))   )
plt.show()

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:8: RuntimeWarning: invalid value encountered in greater


In [16]:
np.savetxt('lmin_indx_valid_Mouse12_120807.csv', lmin_indx_valid, delimiter=';')

# Angles

In [36]:
angles = np.loadtxt('../data/AngFiles/Mouse12-120807.txt')

In [37]:
angles[angles==-1] = np.nan
mask = np.isnan(angles)
angles[mask] = np.interp(  np.flatnonzero(mask) , np.flatnonzero(~mask) , angles[~mask]   )

In [38]:
wind_b, wind_e = 5698.9,7778.8
fr = 39.06
wind_point_b,wind_point_e = int(wind_b*fr), int(wind_e*fr)

In [39]:
angles = angles[wind_point_b:wind_point_e]
angles = np.unwrap(angles)

In [1]:
angles666 = np.abs( angles[30:] - angles[:-30] )
plt.plot( angles666 ) 
plt.show()

NameError: name 'np' is not defined

In [41]:
smooth_angle = savitzky_golay( angles666 , 201, 2) # window size 51, polynomial order 3

In [43]:
plt.plot(angles666)
plt.plot(smooth_angle, 'r', linewidth=3)
plt.show()

In [62]:
local_max = argrelextrema(smooth_angle, np.greater)[0] # for local maxima
local_min = argrelextrema(smooth_angle, np.less)[0] # for local minima

prox_indx = np.where( local_min[1:]-local_min[:-1] > 10 )[0]
prox_indx_valid = local_min[prox_indx]

local_range = 20
lmin_indx = np.where( smooth_angle[prox_indx_valid+local_range][:-local_range] - smooth_angle[prox_indx_valid][:-local_range] > 0.15)[0]
lmin_indx_valid = prox_indx_valid[lmin_indx]

plt.plot( smooth_angle, 'r' , linewidth=3  )
plt.vlines(  lmin_indx_valid , np.zeros(len(lmin_indx_valid)) , np.ones(len(lmin_indx_valid))   )
plt.show()

In [59]:
np.savetxt('lmin_indx_valid_angle.csv', lmin_indx_valid, delimiter=';')